In [5]:
import tensorflow as tf  # The main library for building neural networks
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input  # Parts of the CNN
from tensorflow.keras.models import Model  # To put everything together
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # To handle our pictures
import numpy as np  # For math stuff
import matplotlib.pyplot as plt  # For drawing graphs
import os  # To work with files and folders

2025-03-18 15:37:54.543944: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-18 15:38:07.726308: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [19]:
train_dir='dataset/Train'
test_dir='dataset/test'

IMG_SIZE=(224,224)
BATCH_SIZE=32

In [21]:
train_datagen=ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [22]:
test_datagen=ImageDataGenerator(rescale=1/255)


In [23]:
train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=IMG_SIZE,
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical')

Found 1600 images belonging to 4 classes.


In [24]:
test_generator=test_datagen.flow_from_directory(test_dir,
                                                target_size=IMG_SIZE,
                                                batch_size=BATCH_SIZE,
                                                class_mode='categorical',
                                                shuffle=False)

Found 400 images belonging to 4 classes.


In [25]:
num_classes=len(train_generator.class_indices)

print(f'number of class{num_classes}')
print(f'class label {train_generator.class_indices}')

number of class4
class label {'Healthy': 0, 'Miner': 1, 'Phoma': 2, 'Rust': 3}


In [29]:
inputs=Input(shape=(224,224,3))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x=MaxPooling2D((2,2))(x)

x=Conv2D(64,(3,3),activation='relu',padding='same')(x)
x=MaxPooling2D((2,2))(x)


x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)  


x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)  


x = Flatten()(x)  


x = Dense(512, activation='relu')(x)  
x = Dropout(0.5)(x) 



2025-03-18 16:20:25.939578: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2025-03-18 16:20:26.156945: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2025-03-18 16:20:26.962167: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


In [30]:
outputs=Dense(num_classes,activation='softmax')(x)

In [31]:
model=Model(inputs=inputs,outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 56, 56, 128)       73856 

In [32]:
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [ ]:
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//BATCH_SIZE,
    epochs=30,
    validation_data=test_generator,
    validation_steps=test_generator.samples//BATCH_SIZE,
    callbacks=[early_stopping]
)